## Challenge 3 - Understanding user and product interaction. Part - 2

The goal is: For each city, find the most likely city to be also searched for within the same session.

<span style="color:red">Solution: The safest way to solve that is to calculate the frequency and ratio by each cities relation </span>

In [1]:
from itertools import *
from collections import Counter 
import pandas as pd

### Let's load the cities dataset that was created in the last notebook

In [14]:
with open("data/mysteriours_country.csv") as f:
    content = f.readlines()

# we also want to remove whitespace characters and `\n` at the end of each line
cities_list = [list(map(str.strip,x.split(','))) for x in content] 

flat_list = Counter([item for sublist in cities_list for item in sublist])
flat_list.most_common()

[('New York NY', 754),
 ('Toronto ON', 457),
 ('Los Angeles CA', 344),
 ('Montreal QC', 334),
 ('Chicago IL', 285),
 ('Houston TX', 210),
 ('Vancouver BC', 207),
 ('Philadelphia PA', 145),
 ('Phoenix AZ', 102),
 ('San Diego CA', 101),
 ('Calgary AB', 99),
 ('San Antonio TX', 91),
 ('Jersey City NJ', 88),
 ('OTTAWA ON', 86),
 ('Newark NJ', 86),
 ('Edmonton AB', 74),
 ('Dallas TX', 71),
 ('Indianapolis IN', 66),
 ('Detroit MI', 61),
 ('Jacksonville FL', 60),
 ('San Jose CA', 57),
 ('Columbus OH', 49),
 ('Oshawa ON', 48),
 ('Santa Ana CA', 41),
 ('Victoria BC', 40),
 ('Long Beach CA', 37),
 ('Seattle WA', 35),
 ('Milwaukee WI', 33),
 ('Anaheim CA', 33),
 ('Saint Catharines-Niagara ON', 32),
 ('Austin TX', 27),
 ('Boston MA', 27),
 ('Baltimore MD', 26),
 ('Riverside CA', 23),
 ('Portland OR', 23),
 ('Madison WI', 21),
 ('Kitchener ON', 20),
 ('Fort Wayne IN', 20),
 ('Hamilton ON', 20),
 ('Windsor ON', 19),
 ('Fort Worth TX', 17),
 ('Toledo OH', 17),
 ('Buffalo NY', 15),
 ('Plano TX', 14),


In [11]:
flat_list = Counter([item for sublist in cities_list for item in sublist])
flat_list.most_common()

[('New York NY', 754),
 ('Toronto ON', 457),
 ('Los Angeles CA', 344),
 ('Montreal QC', 334),
 ('Chicago IL', 285),
 ('Houston TX', 210),
 ('Vancouver BC', 207),
 ('Philadelphia PA', 145),
 ('Phoenix AZ', 102),
 ('San Diego CA', 101),
 ('Calgary AB', 99),
 ('San Antonio TX', 91),
 ('Jersey City NJ', 88),
 ('OTTAWA ON', 86),
 ('Newark NJ', 86),
 ('Edmonton AB', 74),
 ('Dallas TX', 71),
 ('Indianapolis IN', 66),
 ('Detroit MI', 61),
 ('Jacksonville FL', 60),
 ('San Jose CA', 57),
 ('Columbus OH', 49),
 ('Oshawa ON', 48),
 ('Santa Ana CA', 41),
 ('Victoria BC', 40),
 ('Long Beach CA', 37),
 ('Seattle WA', 35),
 ('Milwaukee WI', 33),
 ('Anaheim CA', 33),
 ('Saint Catharines-Niagara ON', 32),
 ('Austin TX', 27),
 ('Boston MA', 27),
 ('Baltimore MD', 26),
 ('Riverside CA', 23),
 ('Portland OR', 23),
 ('Madison WI', 21),
 ('Kitchener ON', 20),
 ('Fort Wayne IN', 20),
 ('Hamilton ON', 20),
 ('Windsor ON', 19),
 ('Fort Worth TX', 17),
 ('Toledo OH', 17),
 ('Buffalo NY', 15),
 ('Plano TX', 14),


### Using permutations between cities in the same session create a list of tuples 

In [4]:
# permutations between cities inside lists
cities_permutation = [list(permutations(x, r=2)) for x in cities_list]

# add them to a single list
cities_permutation_list = list(chain.from_iterable(cities_permutation))


### Create a pandas dataframe with the count of duplicated tuples

In [9]:
# Create a pandas dataframe with the count of duplicated tuples
cities_tuples_dictionary = []
for cities, count in Counter(cities_permutation_list).most_common():
    cities_tuples_dictionary.append({"city_1":cities[0], "city_2":cities[1], "n_ocurrences":count})
    

In [10]:
cities_pd = pd.DataFrame(cities_tuples_dictionary).sort_values(['city_1','n_ocurrences'], ascending=False)
cities_pd.head()

,city_1,city_2,n_ocurrences
167,Windsor ON,Detroit MI,64
261,Windsor ON,Toledo OH,37
434,Windsor ON,Cleveland OH,18
455,Windsor ON,Fort Wayne IN,16
475,Windsor ON,Chicago IL,15


### Calculating the ratio of ocurrences

In [26]:
cities_grouped = cities_pd.groupby('city_1').agg({'n_ocurrences': 'sum'}).reset_index()
cities_grouped.columns = ['city_1', 'total_ocurrences']

cities_total_pd = pd.merge(cities_pd, cities_grouped, on='city_1', how='inner')

cities_total_pd['ratio_ocurrences'] = cities_total_pd['n_ocurrences']/ cities_total_pd['total_ocurrences']

### Keep only the top ocurrences by city

In [27]:
# get the most duplicated cities tuple (with most number of ocurrences)
top_cities_pd = cities_total_pd.groupby('city_1').first().reset_index()

# Show the top 10
top_cities_pd.sort_values('n_ocurrences', ascending=False).head(10)

,city_1,city_2,n_ocurrences,total_ocurrences,ratio_ocurrences
50,New York NY,Jersey City NJ,549,4056,0.135355
32,Jersey City NJ,New York NY,549,1202,0.456739
51,Newark NJ,New York NY,435,1100,0.395455
58,Philadelphia PA,New York NY,428,1211,0.353427
80,Toronto ON,New York NY,420,2653,0.158311
39,Los Angeles CA,New York NY,335,2122,0.157870
47,Montreal QC,New York NY,315,1854,0.169903
57,Oshawa ON,Toronto ON,288,721,0.399445
13,Chicago IL,New York NY,270,1657,0.162945
67,Saint Catharines-Niagara ON,Toronto ON,267,681,0.392070
